# Card Game with Augmented Reality

In [137]:
import cv2
import numpy as np
import os
import imutils.perspective

In [138]:
database_folder_path = "./resized"
frame_test_path = "./frames_test/frame_3.jpg"

In [139]:
class Card:
    def __init__(self, name, suit, img) -> None:
        self.__name = name
        self.__suit = suit
        self.__img = img

    def get_name(self):
        return self.__name

    def get_suit(self):
        return self.__suit

    def get_img(self):
        return self.__img

In [140]:
def extract_card_border(card_img):
    border_h = 200
    border_w = 100
    return card_img[0:border_h, 0:border_w]

In [141]:
def create_binary(frame):
    # convert to gray
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # apply blur
    frame = cv2.GaussianBlur(frame, (3,3), 0)

    # convert to binary
    frame = cv2.threshold(frame, 210, 255, cv2.THRESH_BINARY)[1]

    return frame

In [142]:
def extract_card_name_suit(filename):
    #suit_legend = {"C": "Clubs", "D": "Diamonds", "H": "Hearts", "S": "Spades"}
    name = filename.split(".")[0]
    name_split = name.split("_")
    return name_split[0], name_split[1]

def read_all_cards():
    cards = set()

    filenames = os.listdir(database_folder_path)
    for filename in filenames:
        frame = cv2.imread(f"{database_folder_path}/{filename}")
        binary_frame = create_binary(frame)
        name, suit = extract_card_name_suit(filename)
        cards.add(Card(name, suit, binary_frame))

    return cards


In [143]:
def find_contourns(binary):
    cnts = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    #for cnt in cnts:
    #    approx = cv2.convexHull(cnt)
    #    approx = cv2.approxPolyDP(approx,0.01*cv2.arcLength(cnt,True),True)

    return cnts

In [144]:
def find_corners_contourn(cnt, binary_frame):
    # cnt[:, :, 0] -> all x values
    # cnt[:, :, 1] -> all y values
    # cnt[:, :, 0].argmin() -> index min x value
    # cnt[cnt[:, :, 0].argmin()][0] -> pair min x value
    #left_point = tuple(cnt[cnt[:, :, 0].argmin()][0])
    #right_point = tuple(cnt[cnt[:, :, 0].argmax()][0])
    #top_point = tuple(cnt[cnt[:, :, 1].argmin()][0])
    #botton_point = tuple(cnt[cnt[:, :, 1].argmax()][0])
    #print(cnt.shape)
    #print(f"{left_point},{right_point}, {top_point}, {botton_point}")
    #print(cnt)
    #print(cnt[cnt[:, :, 0].argmin()][0])
    #return left_point, right_point, top_point, botton_point


    # https://stackoverflow.com/questions/50984205/how-to-find-corners-points-of-a-shape-in-an-image-in-opencv
    rect = cv2.minAreaRect(cnt)

    binary_frame = np.float32(binary_frame)
    mask = np.zeros(binary_frame.shape, dtype="uint8")
    cv2.fillPoly(mask, [cnt], (255,255,255))
    dst = cv2.cornerHarris(mask,60,7,0.04)
    ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(binary_frame,np.float32(centroids),(5,5),(-1,-1),criteria)
    corners = corners[1:]
    print("Pau")
    print(corners)
    corners = imutils.perspective.order_points(corners)

    return corners



In [145]:
def apply_homography(corners, frame):
    database_width = 500
    database_height = 726
    database_points = np.array([[0, database_height], [0, 0], [database_width, 0], [database_width, database_height]])
    h, status = cv2.findHomography(corners, database_points)
    print("Corners")
    print(corners)
    #warped_frame = cv2.warpPerspective(frame, h, (frame.shape[1], frame.shape[0]))
    warped_frame = cv2.warpPerspective(frame, h, (database_width, database_height))
    return warped_frame

In [146]:
def extract_frame_cards(frame):
    frame_height, frame_width, _ = frame.shape
    card_size_threshold = 0.1
    frame_cards = []
    
    binary_frame = create_binary(frame)
    cnts = find_contourns(binary_frame)

    #corners = find_corners(binary_frame)
    #corners = cv2.dilate(corners, None)
    #frame[corners>0.01*corners.max()]=[0,0,255]
    
    #for i in corners:
    #    x,y = i.ravel()
    #    cv2.circle(frame,(int(x),int(y)),3,(255, 0, 0),-1)

    #left_point, right_point, top_point, botton_point = find_corners_contourn(cnts[0])
    #cv2.circle(frame, left_point, 8, (0, 0, 255), -1)
    #cv2.circle(frame, right_point, 8, (0, 255, 0), -1)
    #cv2.circle(frame, top_point, 8, (255, 0, 0), -1)
    #cv2.circle(frame, botton_point, 8, (255, 255, 0), -1)

    n_valid_cnts = 0
    i = 0
    for cnt in cnts:
        cnt_x, cnt_y, cnt_width, cnt_height = cv2.boundingRect(cnt)
        if (cnt_width < frame_width * card_size_threshold) or (cnt_height < frame_height * card_size_threshold):
            continue
        n_valid_cnts += 1
        #cv2.drawContours(frame,[cnt],-1,(255,0,0),3)
        frame_card = binary_frame[cnt_y:cnt_y+cnt_height, cnt_x:cnt_x+cnt_width]

        corners = find_corners_contourn(cnt, binary_frame)
        #print(corners.shape)
        for corner in corners:
            #print(tuple(corner))
            cv2.circle(frame, (int(corner[0]), int(corner[1])), 8, (0, 0, 255), -1)

        warped_frame_card = apply_homography(corners, binary_frame)
        cv2.imshow(f"{i}", warped_frame_card)

        #corners = find_corners(frame_card)
        #corners = cv2.dilate(corners, None)
        #frame_card_color = frame_card.copy()
        #frame_card_color = cv2.cvtColor(frame_card_color, cv2.COLOR_GRAY2RGB)
        #frame_card_color[corners>0.01*corners.max()]=[0,0,255]
        #cv2.imshow(f"frame card color {i}", frame_card_color)
        #cv2.imshow(f"{i}", warped_frame_card)
        i += 1

        '''
        cnt_rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(cnt_rect)
        box = np.int0(box)

        # get width and height of the detected rectangle
        cnt_width = int(cnt_rect[1][0])
        cnt_height = int(cnt_rect[1][1])

        src_pts = box.astype("float32")
        # coordinate of the points in box points after the rectangle has been
        # straightened
        dst_pts = np.array([[0, cnt_height-1],
                            [0, 0],
                            [cnt_width-1, 0],
                            [cnt_width-1, cnt_height-1]], dtype="float32")

        # the perspective transformation matrix
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)

        # directly warp the rotated rectangle to get the straightened rectangle
        frame_card = cv2.warpPerspective(frame, M, (cnt_width, cnt_height))
        '''

        #frame_cards.append((frame_card, cnt))
        #frame_cards.append((warped_frame_card, cnt))

    print("Valid: " + str(n_valid_cnts))

    

    return frame_cards


In [147]:
def find_matches(frame_card, database_card):
    orb = cv2.ORB_create(nfeatures=2000)
    kp1, des1 = orb.detectAndCompute(frame_card, None)
    kp2, des2 = orb.detectAndCompute(database_card, None)

    
    #bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    bf = cv2.BFMatcher()
    #matches = bf.match(des1, des2)
    matches = bf.knnMatch(des1, des2, k=2)

    '''
    index_params = dict(algorithm=6,
                        table_number=6,
                        key_size=12,
                        multi_probe_level=2)
    search_params = {}
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    '''
    
    # As per Lowe's ratio test to filter good matches
    good_matches = []
    for match in matches:
        if len(match) == 2:
            m, n = match
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

    match_img = cv2.drawMatches(frame_card, kp1, database_card, kp2, good_matches, None)

    #if len(good_matches) > 50:
        #src_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        #dst_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        #m, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
        #corrected_img = cv2.warpPerspective(img1, m, (img2.shape[1], img2.shape[0]))

    return good_matches, match_img


In [148]:
database_cards = read_all_cards()

frame = cv2.imread(frame_test_path)
#frame = cv2.resize(frame, (0,0), fx=0.4, fy=0.4) 
#binary_frame = create_binary(frame)
#cnts = find_contourns(binary_frame)
#cv2.drawContours(frame, cnts, -1, (0,255,0), 3)
frame_cards = extract_frame_cards(frame)
match_imgs = []
for frame_card, cnt in frame_cards:
    l = []
    max_matches = -1
    match_img_to_show = None
    best_database_card = None
    #for database_card in database_cards:
    #    matches, match_img = find_matches(frame_card, database_card.get_img())
    #    if len(matches) > max_matches:
    #        match_img_to_show = match_img
    #        best_database_card = database_card
    #        max_matches = len(matches)
    #    l.append(len(matches))
    
    #match_imgs.append(match_img_to_show)
    #print(max_matches)
    #print(l)
    cv2.drawContours(frame,[cnt],-1,(255,0,0),3)
    x,y,w,h = cv2.boundingRect(cnt)
    #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    #cv2.putText(frame,best_database_card.get_name() + " " + best_database_card.get_suit(),(x+10,y+30), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2)

Pau
[[1681.5891  755.702 ]
 [2406.2124 1002.5155]
 [ 605.9991 1256.0973]
 [1435.7585 1666.0187]]
Corners
[[ 605.9991 1256.0973]
 [1681.5891  755.702 ]
 [2406.2124 1002.5155]
 [1435.7585 1666.0187]]
Pau
[[2688.4343   526.95   ]
 [3871.138    760.49133]
 [2434.934    860.67914]
 [3855.0002  1183.6544 ]]
Corners
[[2688.4343   526.95   ]
 [3871.138    760.49133]
 [3855.0002  1183.6544 ]
 [2434.934    860.67914]]
Pau
[[ 610.41925  278.65964]
 [1561.5211   492.51572]
 [ 149.45137  523.0445 ]
 [1294.7437   774.4951 ]]
Corners
[[ 610.41925  278.65964]
 [1294.7437   774.4951 ]
 [1561.5211   492.51572]
 [ 149.45137  523.0445 ]]
Pau
[[2150.956    230.78537]
 [1603.9401   254.90492]
 [2526.7815   501.95538]
 [1866.2008   575.8453 ]]
Corners
[[1603.9401   254.90492]
 [2150.956    230.78537]
 [2526.7815   501.95538]
 [1866.2008   575.8453 ]]
Valid: 4


In [149]:
cv2.imshow("Frame", frame)
#for i in range(len(match_imgs)):
#    cv2.imshow(f"Match Image {i+1}", match_imgs[i])
cv2.waitKey(0)
cv2.destroyAllWindows()